## Numerical Analysis - Fall semester 2024

# Serie 08 - Numerical integration and linear systems

Importing some useful packages.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import timeit

<hr style="clear:both">

### Adaptive quadrature


We would like to approximate the integral $I(f)=\int_a^bf(x)~\mathrm{d}x$
within a given tolerance $tol$. That is, we want to determine $h$ such as 

$$
|I(f)-Q_h(f)|\le tol,
$$
where $Q_h$ is a quadrature formula of order $p$. As the exact integral $I(f)$ is, in general, unknown, we have to introduce a suitable error estimator. A possible estimator of the error for $Q_{h}(f)$ is

$$
\eta:=\frac{|Q_{h}(f)-Q_{2h}(f)|}{2^p-1},
$$

which uses Richardson's extrapolation. It is based on the fact that

$$
\tilde{Q}_{h}:=\frac{2^pQ_{h}(f)-Q_{2h}(f)}{2^p-1}
$$

is an approximation of $I(f)$ of order (at least) $p+1$, as long as $f$ is regular enough, and therefore

$$
\underbrace{|I(f)-Q_{h}(f)|}_{\mathcal{O}(h^p)}\leq \underbrace{|I(f)-\tilde{Q}_{h}(f)|}_{\mathcal{O}(h^{p+1})}+\underbrace{|\tilde{Q}_{h}(f)-Q_{h}(f)|}_{\mathcal{O}(h^p)}\approx |\tilde{Q}_{h}(f)-Q_{h}(f)|=\eta.
$$

To solve the following exercises, we provide you with the implementation of the midpoint quadrature, which you've already used to solve last week's notebook. Run the below cell to use it later on.

In [ ]:
def midpoint(a, b, n, f):
    # Composite midpoint formula
    #  - a,b: boundaries of the integration interval
    #  - n: number of sub-intervals
    #  - f: function to integrate
    h = (b - a) / n
    xi = np.linspace(a + h / 2, b - h / 2, n)  # quadrature nodes
    alphai = np.full(n, h)  # weights
    Qh_mp = np.dot(alphai, f(xi))  # quadrature formula
    return Qh_mp

<div class="alert alert-success">
    
**Exercise 1:** Implement the function `adaptive_quadrature`, which adaptively approximates the integral $I(f)=\int_a^b f(x)~\mathrm{d}x$ using the midpoint quadrature implemented in the `midpoint` function above. That is, it iteratively doubles the number of sub-intervals `n` of the composite midpoint quadrature, computes the corresponding midpoint quadrature, and computes the error estimate $\eta$, until the error estimate $\eta$ lies below the tolerance.
</div>

In [ ]:
def adaptive_quadrature(a, b, f, tol):
    n = 1  # current number of sub-intervals
    p = 2  # order of midpoint formula

    # lists which should be filled in at every iteration
    Qh_list = [midpoint(a, b, n, f)]  # quadrature approximation for each number of sub-intervals
    h_list = [(b - a) / n]  # h for each number of sub-intervals
    eta = tol + 1  # set the first error estimate to not already meet tolerance

    while eta > tol:
        # YOUR CODE HERE
        raise NotImplementedError()

    # convert lists to numpy arrays for easier manipulations
    return np.array(Qh_list), np.array(h_list)

<div class="alert alert-success">
    
**Exercise 2:** 
Consider the function $f(x)=\frac{\sin(x)\cos^3(x)}{4-\cos^2(2x)}$, on the interval $[a,b]=[0,\pi/2]$. Using the adaptive quadrature you have just implemented based on the error estimate $\eta$, determine (numerically) the number $n=(b-a)/h$ of sub-intervals necessary to obtain an approximation of the integral with an estimated error lower than $10^{-5}$.
</div>


In [ ]:
def f(x):
    # YOUR CODE HERE
    raise NotImplementedError()

# YOUR CODE HERE
raise NotImplementedError()

<div class="alert alert-success">
    
**Exercise 3:** 
Knowing that the exact value of the integral is $I(f)=\log(3)/16$, plot the graph of the estimator and of the "true" error in terms of $h$ in logarithmic scale (command `plt.loglog`). What can you say about the order of convergence and the quality of the error estimator?
</div>


In [ ]:
I_f = np.log(3) / 16

# YOUR CODE HERE
raise NotImplementedError()

<div class="alert alert-success">
    
**Exercise 4:** 
On the same graph as in Exercise 3, plot the error $|I(f) - \tilde{Q}_h|$ in terms of $h$, where $\tilde{Q}_h$ is the Richardson's extrapolation based on $Q_h$ and $Q_{2h}$. What order of convergence do you observe in this case?
</div>


In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

<div class="alert alert-success">
    
**Exercise 5:** 
Repeat all the previous exercises in this section for $f(x)=\sqrt{|x|^3}$ and $[a,b]=[-2,2]$, the exact value of the integral being in this case $I(f)=16\sqrt{2}/5$. Comment the obtained results.
</div>


In [ ]:
I_f = 16 * np.sqrt(2) / 5

# YOUR CODE HERE
raise NotImplementedError()

<div class="alert alert-success">
    
**Exercise 6:** 
Repeat all the previous exercises in this section for $f(x)=\log(x)$ and $[a,b]=[0,1]$, the exact value of the integral being in this case $I(f)=-1$. Comment the obtained results.
</div>

In [ ]:
I_f = -1

# YOUR CODE HERE
raise NotImplementedError()

<hr style="clear:both">

### Simple linear systems

Consider the following two linear systems:

$$
  A\mathbf{x} = \mathbf{b}_1, \qquad \text{and} \qquad
  A= \begin{pmatrix}
          2 & 4 & 8 \\
          1 & 1 & 4 \\
          3 & 6 & 7
        \end{pmatrix}, \quad
  \mathbf{b}_1 = \begin{pmatrix} 6 \\ 5 \\ 4 \end{pmatrix},
$$

and

$$
  B\mathbf{x} = \mathbf{b}_2, \qquad \text{and} \qquad
  B = \begin{pmatrix}
          1 & 1 & 1 & 1 \\
          2 & 2 & 5 & 3 \\
          4 & 6 & 8 & 0 \\
          3 & 3 & 9 & 8
        \end{pmatrix}, \quad
  \mathbf{b}_2 = \begin{pmatrix} 1 \\ 2 \\ 5 \\ 0 \end{pmatrix}.
$$

<div class="alert alert-success">
    
**Exercise 7 (Theoretical):** Compute the LU factorization of the matrix $A$ by hand.
</div>


<div class="alert alert-success">
    
**Exercise 8 (Theoretical):** Solve the linear system $A \mathbf{x} = \mathbf{b}_1$ using the factorization found in the previous exercise.
</div>


<div class="alert alert-success">
    
**Exercise 9 (Theoretical):** Check that the algorithm of the LU factorization without pivoting for the matrix $B$ cannot be executed until the end.
</div>



<div class="alert alert-success">
    
**Exercise 10 (Theoretical):** Find a permutation matrix $P$ by rows such as $PB$ is factorisable, then calculate the LU factorization of $PB$.
</div>



<div class="alert alert-success">
    
**Exercise 11 (Theoretical):** Solve the linear system $B\mathbf{x} = \mathbf{b}_2$ using the factorization previously found.
</div>


<div class="alert alert-success">
    
**Exercise 12 (Theoretical):** Compute the determinant of the matrix $B$ using its LU factorization.

*Hint:* We know that
$$
\text{det}(B)=\text{det}(P^{-1}LU)=\displaystyle\frac{\text{det}(L) \text{det}(U)}{\text{det}(P)}.
$$
</div>


<hr style="clear:both">

### LU factorization for dense and sparse matrix 

<div class="alert alert-success">
    
**Exercise 13:** Complete the function `binomial_matrix` which creates a matrix $A \in \mathbb{R}^{m \times m}$ whose entries $a_{ij}$ for $i,j = 1, 2, \dots, m$ independently follow a binomial distribution with $n \in \mathbb{N}$ trials and a success probability of $p \in [0, 1]$. Use your implementation to create the $(n=10, p=0.5)$-binomial matrix $A \in \mathbb{R}^{400 \times 400}$. Visualize its non-zero entries using the function `plt.spy(A)`, which plots the non-zero entries of a matrix in black.

*Hint:* The NumPy function `np.random.binomial(n, p, (m, m))` outputs an $m \times m$ NumPy array whose entries are independently $(n, p)$-binomial distributed. The seed of NumPy's random number generator is fixed to `seed=0`, such that you will always get the same random matrix for the same parameters.
</div>

In [ ]:
def binomial_matrix(m, n, p, seed=0):
    np.random.seed(seed)
    # YOUR CODE HERE
    raise NotImplementedError()

# YOUR CODE HERE
raise NotImplementedError()

<div class="alert alert-success">
    
**Exercise 14:** Compute the LU factorization using the function `sp.linalg.lu`, which takes as an input a matrix $A$ and returns the three matrices $P$, $L$, and $U$ which characterize the LU factorization of $A$. Visualize the non-zero entries of $L$, $U$, and $P$ (again use `plt.spy`). Have row-permutations been performed? How do the number of non-zero elements in $L$ and $U$ compare with what we saw for $A$?
</div>

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

<div class="alert alert-success">
    
**Exercise 15:** For the binomial matrix $A$ with dimensions $m=20^2, 21^2,\ldots,35^2$, compute the number of non-zero entries $\text{nnz}(A)$ in $A$ and compare it to the number of non-zero elements $\text{nnz}(L) + \text{nnz}(U)$ in its LU factors. Plot the results in a logarithmic plot, and add the number of entries $m^2$ of the full matrix for comparison.

*Hint:* Use `np.count_nonzero` to count the number of non-zero entries in a NumPy array.
</div>

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Below, we provide you with a function which computes the runtime of the LU factorization algorithm `sp.linalg.lu` for any matrix `A` by executing the function on the matrix multiple times, and averaging the individual runtimes. The number of repetitions of the function execution can be controlled with the argument `repeats`.

In [ ]:
def lu_runtime(A, repeats):
    t = timeit.timeit(lambda: sp.linalg.lu(A), number=repeats) / repeats
    return t

<div class="alert alert-success">
    
**Exercise 16:** Measure the approximate runtime for computing the LU factorization of the binomial matrix $A$ of dimensions $m=20^2, 21^2,\ldots,35^2$. Set `repeats=50` to average over $50$ executions of the algorithm to get a more stable estimate of the true run-time.
</div>

<div class="alert alert-info">

**Note:** It may happen that the runtime of the first $m$ is significantly higher. This is due to the caching which Python performes in the background. If it bothers you, you may simply ignore the first $m$.
</div>

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

The following function `laplacian_matrix` generates a Laplacian matrix $B \in \mathbb{R}^{m \times m}$.

In [ ]:
def laplacian_matrix(m):
    n = int(np.sqrt(m))
    d = np.ones(n ** 2)
    mat = sp.sparse.spdiags([d, -2 * d, d], [-1, 0, 1], n, n)
    I = sp.sparse.eye(n)
    A = sp.sparse.kron(I, mat) + sp.sparse.kron(mat, I) 
    return A.todense()

<div class="alert alert-success">
    
**Exercise 17:** Repeat all of the above exercises in this section for the Laplacian matrix $B$ with the same dimensions. Compare to the results you've obtained for the binomial matrix $A$.
</div>

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

<hr style="clear:both">

## The end

Congratulations! This is the end of the eighth exercise notebook.